In [1]:
# pip install pdfplumber

In [2]:
# ! pip install -q memory_profiler

In [3]:
# pip install -U gensim

In [4]:
# pip install nltk

In [5]:
import pdfplumber
pdf = pdfplumber.open('A brief History of Modern India - Rajiv Ahir.pdf')
c = 0
pdf_txt = ""
flag = True
while flag == True:
    try:
        # print(c)
        page = pdf.pages[c]
        c = c+1
        pdf_txt = pdf_txt + page.extract_text()
    except:
        pdf.close()
        flag = False

In [6]:
print(pdf_txt[:100])

t.me/Ebooks_Encyclopedia27. t.me/Magazines4all
A Brief History
of Modern India
by
Rajiv Ahir
I.P.S.



In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ashishsingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    sentence = re.sub(r'\n', ' ', sentence)
    if stopwords:
        sentence = remove_stopwords(sentence)
    return sentence

def get_cleaned_sentences(tokens, stopwords=False):
    cleaned_sentences = []
    for row in tokens:
        cleaned = clean_sentence(row, stopwords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

In [9]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np

def load_model():
    v2w_model = None
    try:
        v2w_model = KeyedVectors.load('./w2vecmodel.mod')
        print("w2v Model Successfully loaded")
    except:
        v2w_model = api.load('word2vec-google-news-300')
        v2w_model.save("./w2vecmodel.mod")
        print("w2v Model Saved")

    w2vec_embedding_size = len(v2w_model['pc'])
    
    return v2w_model, w2vec_embedding_size

In [10]:
def getWordVec(word, model):
    samp = model['pc']
    vec = [0]*len(samp)
    try:
        vec = model[word]
    except:
        vec = [0]*len(samp)
    return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
    samp = getWordVec('computer', embeddingmodel)
    vec = np.array([0]*len(samp))
    den = 0;
    for word in phrase.split():
        den = den+1
        vec = vec+np.array(getWordVec(word, embeddingmodel))
    return vec.reshape(1, -1)

In [11]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
    max_sim = -1
    index_sim = -1
    for index, embedding in enumerate(sentence_embeddings):
        sim = cosine_similarity(embedding, question_embedding)[0][0]
        # print(index, sim, sentences[index])
        if sim > max_sim:
            max_sim = sim
            index_sim = index
  
    return index_sim

In [12]:
def train_predict_pipeline(question):
    raw_sentences = nltk.sent_tokenize(pdf_txt)
    cleaned_sentences = get_cleaned_sentences(raw_sentences, stopwords=True)
    cleaned_sentences_with_stopwords = get_cleaned_sentences(raw_sentences, stopwords=False)
    
    v2w_model, w2vec_embedding_size = load_model()
    
    sent_embeddings = []
    for sent in cleaned_sentences:
        sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))
    
    question = clean_sentence(question, stopwords=True)
    question_embedding = getPhraseEmbedding(question, v2w_model)
    index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)
    
    print("Question: ", question)
    print("Answer: ", cleaned_sentences_with_stopwords[index])

In [17]:

question = "Which pilot led the Vasco Da Gama ships ?"

%time train_predict_pipeline(question)

w2v Model Successfully loaded
Question:  pilot led vasco da gama ships
Answer:  from trading to ruling vasco da gama the arrival of three ships under vasco da gama led by a gujarati pilot named abdul majid at calicut in may 1498 profoundly affected the course of indian history
CPU times: user 2.73 s, sys: 1.32 s, total: 4.04 s
Wall time: 4.93 s


In [18]:
%reload_ext memory_profiler
%memit train_predict_pipeline(question)

w2v Model Successfully loaded
Question:  pilot led vasco da gama ships
Answer:  from trading to ruling vasco da gama the arrival of three ships under vasco da gama led by a gujarati pilot named abdul majid at calicut in may 1498 profoundly affected the course of indian history
peak memory: 3867.81 MiB, increment: 3301.19 MiB


In [16]:

question = "What is Marxist Approach ?"

%time train_predict_pipeline(question)

w2v Model Successfully loaded
Question:  marxist approach
Answer:  marxist historiography approach the beginning of the marxist approach in india was heralded by two classic booksrajni palme dutts india today and ar
CPU times: user 2.74 s, sys: 1.25 s, total: 4 s
Wall time: 5.24 s
